# ExtendedProfile — baseline A (estático) / opción B (rolling)

In [1]:
# Imports\n# Ensure project root is on sys.path so `import src.*` works when running from notebooks/
import sys
from pathlib import Path

project_root = Path.cwd().parent  # notebooks/ -> project root
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

import pandas as pd
import numpy as np
from pathlib import Path
import yaml

from src.data_io import load_market_data
from src.sessions import infer_sessions
from src.ep_builder import build_ep
from src.scoring import score_levels, filter_levels, crowd_density, crowd_aware_score
from src.clustering import cluster_levels
from src.metrics import hue, hazard_first_touch, contrast_lift, isolation_from_density, reactivity_index
from src.utils import to_ticks, from_ticks, zscore


In [2]:
# Cargar params
params_path = Path("../configs/params.yml")
with open(params_path, 'r', encoding='utf-8') as f:
    P = yaml.safe_load(f)
P


{'start_date': '2025-01-01',
 'end_date': '2025-02-28',
 'instrument': 'ES',
 'timeframe': '5m',
 'tick_size': 0.25,
 'ep_mode': 'static',
 'rolling_bars': 1000,
 'n_prev_sessions': 4,
 'price_range': None,
 'filter': {'method': 'percentile', 'value': 95, 'topN': 12},
 'cluster': {'merge_ticks': 1},
 'horizon': 'next_session',
 'metrics': {'X_touch_ticks': 1,
  'K_density_ticks': 3,
  'M_contrast_ticks': 8,
  'refractory_min': 10,
  'decay': 0.6},
 'weights_init': {'High': 0.25,
  'Low': 0.25,
  'Close': 0.25,
  'MVC': 0.2,
  'UpperWick': 0.03,
  'LowerWick': 0.02},
 'use_volume': True,
 'volume_feats': ['VolAtLevel', 'VPGradient', 'MVC_anchor'],
 'volume_transform': 'log1p_zscore',
 'wv_init': 0.25,
 'crowding_beta': 0.5,
 'purity_gamma': 0.2,
 'optimize': {'n_trials': 50,
  'study_direction': 'maximize',
  'constraints': {'use_close_only': True,
   'max_beacons_per_session': 12,
   'min_beacons_per_session': 3}}}

## 1) Carga de datos (ajusta la ruta a tu fichero Parquet/CSV de ES 5m)

In [3]:
# TODO: ajusta esta ruta a tus datos locales
data_path = Path("../data/ES_5m.parquet")  # o CSV

df = load_market_data(data_path, P['start_date'], P['end_date'])
df = infer_sessions(df)
df.head()


FileNotFoundError: [Errno 2] No such file or directory: '..\\data\\ES_5m.parquet'

## 2) Selección de sesión objetivo y EP de 4 previas

In [ ]:
# Tomamos la última sesión disponible del rango y construimos EP con 4 previas
tick = float(P['tick_size'])
target_session = int(df['SessionId'].max())
ep = build_ep(df, session_id=target_session, n_prev_sessions=int(P['n_prev_sessions']), tick_size=tick)
ep.head()


## 3) Score, densidad (crowd-aware) y balizas

In [ ]:
# Pesos iniciales
w = P['weights_init']

scored = score_levels(ep, w)
dens = crowd_density(scored, K=int(P['metrics']['K_density_ticks']))
# CP: por ahora placeholder 0.0 (lo implementaremos cuando contemos familias activas por nivel)
cp = 0.0
scored_star = crowd_aware_score(scored, dens, cp, beta=float(P['crowding_beta']), gamma=float(P['purity_gamma']))

# Filtrado (percentil/topN)
flt_method = P['filter']['method']
flt_value = P['filter']['value'] if flt_method == 'percentile' else P['filter']['topN']
levels = filter_levels(scored_star, method=flt_method, value=flt_value)

# Clustering
clusters = cluster_levels(levels, merge_ticks=int(P['cluster']['merge_ticks']))
levels.head(), clusters.head()


## 4) Métricas de reactividad en la sesión objetivo

In [ ]:
# Placeholder: aquí deberíamos calcular hits/exposición por nivel durante la sesión target.
# De momento, creamos estructuras vacías para que el notebook sea ejecutable.
hue_series = pd.Series(dtype='float64', name='HUE')
ri_rows = []
for p in levels.index:
    HUEp = float(hue_series.get(p, 0.0))
    lam1 = 0.0
    CP = 0.0
    CL = 0.0
    I  = float(np.exp(-dens.get(p, 0.0))) if p in dens.index else 1.0
    RI = reactivity_index(HUEp, lam1, CP, CL, I)
    ri_rows.append({'level_ticks': int(p), 'HUE': HUEp, 'lambda1': lam1, 'CP': CP, 'CL': CL, 'I': I, 'RI': RI})
ri_df = pd.DataFrame(ri_rows).set_index('level_ticks')
ri_df.head()


## 5) TODOs inmediatos

- Implementar cálculo de **hits** y **exposición** por nivel (±X ticks) durante la **sesión objetivo**.
- Implementar **Contrast Lift** con vecindad ±M, y **CP** (pureza de confluencia).
- Añadir soporte **rolling** (EP dinámico) con ventana `rolling_bars` y anti-lookahead.
- Visualizaciones: heatmap EP, top-N por Score*, y distribución de RI.
- Bench A/B: estático vs rolling; con y sin volumen.
